<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/Toshkentdagi_uylarni_baholash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

## Ma'lumotlarni ixchamlashtirish

In [5]:
# Kerakli modullarni va ma'lumotlarni o'qib olamiz
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn

# Onlayn dataset manzilini ko'rsatamiz
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')

# location ustunidagi ma'lumotlarni alohida qilamiz kvartalni ajratib olib yangi ustun sifatida tekshirib ko'ramiz
# Splitting the 'location' column by commas and creating new columns
# split_columns = df['location'].str.split(',', expand=True)
# # Selecting only the relevant columns (first three columns)
# split_columns = split_columns.iloc[:, :3]
# # Assigning column names
# split_columns.columns = ['city', 'district_loc', 'street']
# # Concatenating the split columns with the original DataFrame
# df = pd.concat([df, split_columns], axis=1)
# Dropping the original 'location' column city va district_loc ustunlariga ehtiyoj bo'lmaganligi uchun tashlab yuboramiz
# df.drop(['city', 'district_loc','location'], axis=1, inplace=True)

# Calculate the IQR for every column
def df_iqr(df_new, column, high=0.95, low=0.05):
    room_iqr = df_new[column].quantile(high) - df_new[column].quantile(low)

    # Define lower and upper bounds
    lower_bound = df_new[column].quantile(low) - 1.5 * room_iqr
    upper_bound = df_new[column].quantile(high) + 1.5 * room_iqr
    mask = (df_new[column] < lower_bound) | (df_new[column] > upper_bound)
    count = len(df_new[mask])
    if count!=0:
        print(f"{column}: {count}")
    # Filter the dataframe based on the IQR range for each 'column'
    filtered_df = df_new[~mask]
    return filtered_df

df.drop('location', axis=1, inplace=True)

#sonlik qiymatlar formatini to'g'rillab, orasida noodatiy qiymatlarni nullga o'zgartiramiz
df['size'] = pd.to_numeric(df['size'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Narxi 10000 dan kichiklarini tashlab yuboramiz
df.drop(df[(df['price'] < 10000)|(df['price'] > 174000)].index, inplace=True)

# Maydoni 225 m kv dan kattalarini va 40 m kv dan kichiklarini tashlab yuboramiz
df.drop(df[(df['size'] > 225) | (df['size'] < 40)].index, inplace=True)

# xonalari 6 tadan katta, 10 qavatdan baland va 2 qavatdan kichik uylarni tashlab yuboramiz
df.drop(df[(df.rooms>6) | (df.max_levels>10) | (df.level>10) | (df.max_levels<2)].index, inplace=True)

df = df_iqr(df, 'size', high=0.9, low=0.05)
df = df_iqr(df, 'price', high=0.9, low=0.05)

df.dropna(subset=['size'], inplace=True)
df.dropna(subset=['price'], inplace=True)

# Ma'lumotlarni train (80 %) va test (20 %)ga ajratamiz
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

#Ishlash uchun oddiy (muvozanatsiz) train/test setni tanlaymiz
X_train = train_set.drop('price', axis=1)
y = train_set['price'].copy()

#sonlik qiymatlar formatini to'g'rillab, orasida noodatiy qiymatlarni nullga o'zgartiramiz
# X_train['size'] = pd.to_numeric(X_train['size'], errors='coerce')
# y['price'] = pd.to_numeric(y['size'], errors='coerce')

X_num = X_train.drop('district', axis=1) # raqamli bo'lmagan qiymatlarni tashlab yuborib alohida o'zgaruvchiga yuklaymiz

size: 15


###Pipeline yaratish

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

In [6]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
# cat_attribs = ['district', 'street']
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

X_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_prepared

array([[-1.81646036, -1.09712817, -0.31468258, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75561902, -0.58446616,  0.16850499, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75561902, -0.93938601, -1.28105772, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.36606366,  0.99295542, -0.31468258, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75561902, -0.70277278, -1.28105772, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75561902, -0.89995047,  0.65169255, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
num_attribs

['rooms', 'size', 'level', 'max_levels']

In [ ]:
cat_attribs

['district']

## 4-QADAM Machine Learning

###Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()
LR_model.fit(X_prepared, y)

LinearRegression()

Model natijasini testlash

In [ ]:
test_data = X_train.sample(10)
test_data

,district,rooms,size,level,max_levels
2577,Юнусабадский,3,84.0,9,9
3430,Юнусабадский,3,105.0,5,6
2671,Учтепинский,1,43.0,3,4
4551,Мирабадский,4,95.0,6,9
4284,Мирзо-Улугбекский,3,80.0,4,4
1323,Яккасарайский,1,42.0,2,5
3441,Чиланзарский,2,65.0,9,9
6705,Мирзо-Улугбекский,4,94.0,7,9
6220,Учтепинский,3,75.0,3,9
1459,Учтепинский,2,60.0,5,5


In [ ]:
test_labels = y.loc[test_data.index]
test_labels

2577     55000.0
3430     66455.0
2671     27000.0
4551     70000.0
4284     37000.0
1323     36000.0
3441     35000.0
6705    125000.0
6220     51500.0
1459     36000.0
Name: price, dtype: float64

In [ ]:
test_data_prepared = full_pipeline.transform(test_data)
predicted_prices = LR_model.predict(test_data_prepared)

In [ ]:
predicted_prices

array([56912.11312466, 81122.0150782 , 29069.94752788, 84196.0911996 ,
       59565.05648777, 42736.15187931, 40619.27269147, 66810.51791408,
       55253.7384176 , 38009.05125459])

In [ ]:
# Asl qiymat va taxmin qilingan qiymatni solishtiramiz
pd.DataFrame({'Actual': test_labels, 'Predicted': predicted_prices})

,Actual,Predicted
2577,55000.0,56912.113125
3430,66455.0,81122.015078
2671,27000.0,29069.947528
4551,70000.0,84196.091200
4284,37000.0,59565.056488
1323,36000.0,42736.151879
3441,35000.0,40619.272691
6705,125000.0,66810.517914
6220,51500.0,55253.738418
1459,36000.0,38009.051255


## 5-QADAM. Modelni baholash

In [ ]:
test_set

,district,rooms,size,level,max_levels,price
7431,Учтепинский,2,42.0,4,5,28500.0
4378,Чиланзарский,4,100.0,3,4,56000.0
3571,Учтепинский,3,70.0,5,5,41500.0
4258,Мирзо-Улугбекский,4,93.0,3,5,62000.0
1855,Олмазорский,2,50.0,1,9,37000.0
...,...,...,...,...,...,...
73,Мирзо-Улугбекский,3,95.0,5,5,50000.0
6530,Мирзо-Улугбекский,2,50.0,4,4,26500.0
3448,Чиланзарский,2,50.0,4,5,33500.0
350,Мирзо-Улугбекский,2,48.0,1,4,34499.0


In [9]:
X_test = test_set.drop('price', axis=1)
y_test = test_set['price'].copy()

In [10]:
X_test_prepared = full_pipeline.transform(X_test)

In [20]:
y_predicted = LR_model.predict(X_test_prepared)

In [ ]:
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

16505.164917797134


In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_predicted)
mae

11425.603717266558

##Random Forest modeli


In [12]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

y_predicted = RF_model.predict(X_test_prepared)

In [14]:
#Absolyut kvadaratik xatolik
from sklearn.metrics import mean_squared_error
rf_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
rf_rmse = np.sqrt(rf_mse)
print(rf_rmse)

14280.285581506596


In [15]:
#Absolyut xatolik
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_predicted)
mae

8978.641999154972

## 5-QADAM Cross Validation Modelni baholash

In [16]:
X = df.drop('price', axis=1)
y = df['price'].copy()

X_prepared = full_pipeline.fit_transform(X)

In [17]:
from sklearn.model_selection import cross_val_score

mse_scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-mse_scores)

In [18]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [12635.40923849 13496.8155973  16259.14698581 15522.28017594
 16482.01633694 14724.05009455 14895.75499526 12927.75168896
 13273.30160486 13915.48625689]
Mean: 14413.201297499352
Standard deviation: 1303.8792744391988


##Linear Regression model xatoligi

In [21]:
scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)
display_scores(rmse_scores)

Scores: [14865.92992035 14704.86397869 18333.38753546 17000.04154286
 18187.70614333 17908.2082115  16323.63441658 14901.19486937
 14925.78324109 16133.20441709]
Mean: 16328.395427633664
Standard deviation: 1387.1175483343386


## 6-QADAM. Taqdimot. Modelni saqlab olish

### pickle

In [22]:
import pickle

filename = 'RF_model.pkl'
with open(filename, 'wb') as file:
  pickle.dump(RF_model, open(filename, 'wb'))

#### modelni o'qish

In [23]:
with open(filename, 'rb') as file:
  loaded_model = pickle.load(file)

###joblib - ma'lumotlarni siqib bead va ishlatishga qulay

In [24]:
import joblib

filename = 'LR_model.pkl'
joblib.dump(RF_model, filename)

['LR_model.pkl']

#### modelni o'qish

In [25]:
model = joblib.load(filename)